# Testing Non-Linear Products Valuation

This notebook walks through:

1. Loading historical index fixings  
2. Building a simple two-index yield curve  
3. Building **classic** SABR surfaces (no product split)  
4. Pricing IBOR caplets/floors and swaptions using classic Hagan SABR  
5. Building **top-down** SABR surfaces for SOFR (CAPLET vs SWAPTION)  
6. Pricing RFR caplets/swaptions using top-down SABR  


# 1. Imports & Setup

In [1]:
import pandas as pd
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.sabr import SabrModel
from fixedincomelib.valuation import IndexManager, ValuationEngineRegistry
from fixedincomelib.product import (
    ProductIborCapFloorlet, ProductOvernightCapFloorlet,
    ProductIborCapFloor,   ProductOvernightCapFloor,
    ProductIborSwaption,   ProductOvernightSwaption
)
from fixedincomelib.analytics import SABRCalculator
from fixedincomelib.data import DataCollection, Data1D, Data2D, build_yc_data_collection
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.sabr import valuation_engine_sabr

print("Setup complete.")

Setup complete.


## 2) Load Index Fixings

On first call, `IndexManager.instance()` will read **`fixing/fixings.csv`** and populate its in-memory registry.

In [2]:
IndexManager.instance()

## 3) Build Dummy Yield Curve

We use two indices (SOFR-1B and USD-LIBOR-BBA-1M) with piecewise-constant interpolation.


In [3]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [4]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [5]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"
}]


In [6]:
value_date = "2025-09-24"

In [7]:
yc = YieldCurve(value_date, dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


## 4) Build Dummy SABR Model

We’ll calibrate a flat SABR surface for USD-LIBOR-BBA-1M so that our caplets can be priced.

In [8]:
sabr_libor = pd.DataFrame([
    ["USD-LIBOR-BBA-1M", 0.25, 0.25, 0.015, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 0.25, 1.00, 0.017, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 1.00, 0.25, 0.018, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 1.00, 1.00, 0.020, 0.5, 0.2, -0.3],
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

sabr_sofr = pd.DataFrame([
    ["SOFR-1B", 0.25, 0.25, 0.010, 0.5, 0.2, -0.1],
    ["SOFR-1B", 0.25, 1.00, 0.012, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 0.25, 0.013, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 1.00, 0.015, 0.5, 0.2, -0.1],
], columns=sabr_libor.columns)

sabr_data = pd.concat([sabr_libor, sabr_sofr], ignore_index=True)

In [9]:
for idx_name, sub in sabr_data.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub
            .pivot(index="AXIS1", columns="AXIS2", values=param)
            .sort_index(axis=0).sort_index(axis=1)
        )
        d2 = Data2D.createDataObject(
            data_type       = param.lower(),   # e.g. "normalvol"
            data_convention = idx_name,
            df              = pivot
        )
        data_objs.append(d2)

# 6) Rebuild registry & classic‐SABR model
dc = DataCollection(data_objs)

In [10]:
classic_build_methods = [
    {
      "TARGET":        idx,
      "VALUES":     param,
      "INTERPOLATION": "LINEAR",
      "SHIFT":         0.0,
      "VOL_DECAY_SPEED": 0.2
    }
    for idx in sabr_data["INDEX"].unique()
    for param in ["NORMALVOL","BETA","NU","RHO"]
]

classic_sabr = SabrModel.from_curve(
    valueDate             = value_date,
    dataCollection        = dc,
    buildMethodCollection = classic_build_methods,
    ycModel               = yc
)
print("Classic SABR components:", classic_sabr.components.keys())


Classic SABR components: dict_keys(['USD-LIBOR-BBA-1M-NORMALVOL', 'USD-LIBOR-BBA-1M-BETA', 'USD-LIBOR-BBA-1M-NU', 'USD-LIBOR-BBA-1M-RHO', 'SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


## 5) Price with Classic SABR

In [11]:
caplet_ibor = ProductIborCapFloorlet(
    startDate   ="2025-07-01",
    endDate     ="2025-10-01",
    index       ="USD-LIBOR-BBA-1M",
    optionType  ="CAP",
    strike      =0.02,
    notional    =1_000_000,
    longOrShort ="LONG"
)

ve1 = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},
    caplet_ibor
)
ve1.calculateValue()
print("IBOR Caplet PV (classic):", ve1.value)

AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

In [ ]:
swaption_ibor = ProductIborSwaption(
    optionExpiry="2025-12-01",
    swapStart   ="2026-01-01",
    swapEnd     ="2031-01-01",
    frequency   ="3M",
    iborIndex   ="USD-LIBOR-BBA-1M",
    strikeRate  =0.0175,
    notional    =1_000_000,
    longOrShort ="LONG",
    optionType  ="PAYER"
)

ve2 = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},
    swaption_ibor
)
ve2.calculateValue()
print("IBOR Swaption PV (classic):", ve2.value)

IBOR Swaption PV (classic): ['USD', np.float64(21946381354.200516)]


In [ ]:
swaption_ibor = ProductIborSwaption(
    optionExpiry="2025-12-01",
    swapStart   ="2026-01-01",
    swapEnd     ="2031-01-01",
    frequency   ="3M",
    iborIndex   ="USD-LIBOR-BBA-1M",
    strikeRate  =0.0175,
    notional    =1000000,
    longOrShort ="LONG",
    optionType  ="PAYER"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},
    swaption_ibor
)
ve.calculateValue()
print("IBOR Swaption PV:", ve.value)

IBOR Swaption PV: ['USD', np.float64(21946381354.200516)]


## 6) Build Top-Down SABR Model for SOFR (CAPLET vs SWAPTION)


In [12]:
sabr_sofr_cap = pd.DataFrame([
    ["SOFR-1B", 0.25, 0.083333, 0.0090,    0.5,  0.20, -0.15],
    ["SOFR-1B", 0.25, 0.25,     0.0100,    0.5,  0.20, -0.15],
    ["SOFR-1B", 1.00, 0.083333, 0.0110,    0.5,  0.20, -0.15],
    ["SOFR-1B", 1.00, 0.25,     0.0120,    0.5,  0.20, -0.15],
], columns=sabr_sofr.columns)
sabr_sofr_cap["PRODUCT"] = "CAPLET"

sabr_sofr_sw = pd.DataFrame([
    ["SOFR-1B",0.25,1.0,0.012,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,2.0,0.013,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,3.0,0.014,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,4.0,0.015,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,1.0,0.013,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,2.0,0.014,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,3.0,0.015,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,4.0,0.016,0.5,0.2,-0.1]
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

sabr_sofr_sw["PRODUCT"] = "SWAPTION"

full_sabr_sofr = pd.concat([sabr_sofr_cap, sabr_sofr_sw], ignore_index=True)

data_objs = []

for idx_name, sub in full_sabr_sofr.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub
            .pivot(index="AXIS1", columns="AXIS2", values=param)
            .sort_index(axis=0).sort_index(axis=1)
        )
        data_objs.append(
          Data2D.createDataObject(param.lower(), idx_name, pivot)
        )

topdown_build_methods = []
for product in ("CAPLET","SWAPTION"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        topdown_build_methods.append({
            "TARGET":           "SOFR-1B",
            "VALUES":           param,
            "AXIS1":            "AXIS1",
            "AXIS2":            "AXIS2",
            "INTERPOLATION":    "LINEAR",
            "SHIFT":            0.0,
            "VOL_DECAY_SPEED":  0.2,
            "PRODUCT":          product
        })

dc = DataCollection(data_objs)
td_sabr = SabrModel.from_curve(
    valueDate             = value_date,
    dataCollection        = dc,
    buildMethodCollection = topdown_build_methods,
    ycModel               = yc
)

print("Top-down SABR components:", td_sabr.components.keys())

Top-down SABR components: dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


## 7) Price Top-Down RFR Products


In [13]:
# 7a) Overnight caplet (top-down SABR)
caplet_ois = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1_000_000,
    longOrShort  ="LONG"
)

ve3 = ValuationEngineRegistry().new_valuation_engine(
    td_sabr,
    {"SABR_METHOD":"top-down"},
    caplet_ois
)
ve3.calculateValue()
print("Overnight Caplet PV (top-down):", ve3.value)

Overnight Caplet PV (top-down): ['USD', np.float64(5375.221325685004)]


## 8) Valuation: Overnight Cap/Floor Wrapper


In [14]:
ois_cap = ProductOvernightCapFloor(
    effectiveDate="2026-07-01",
    maturityDate ="2027-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="FLOOR",
    strike       =0.016,
    notional     =1000000,
    longOrShort  ="SHORT"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    td_sabr,
    {"SABR_METHOD": "top-down"},
    ois_cap
)
ve.calculateValue()
print("Overnight Cap/Floor PV:", ve.value)


Overnight Cap/Floor PV: ['USD', np.float64(-145.75642375396876)]


# 8) Price RFR caplets using bottom-up SABR

In [15]:
corr_data = [
    [0.25, 0.25, 0.80],
    [0.25, 0.50, 0.78],
    [0.25, 1.00, 0.75],
    [1.00, 0.25, 0.78],
    [1.00, 0.50, 0.76],
    [1.00, 1.00, 0.74],
]
df_corr = pd.DataFrame(corr_data, columns=["EXPIRY","TENOR","CORR"])
df_corr["INDEX"] = "SOFR-1B"

In [16]:
bu_objs = []
corr_pivot = (
    df_corr
    .pivot(index="EXPIRY", columns="TENOR", values="CORR")
    .sort_index(axis=0).sort_index(axis=1)
)
bu_objs.append(
    Data2D.createDataObject("corr", "SOFR-1B", corr_pivot)
)

In [17]:
one_bd = 1.0/252

sabr_sofr_1bd = pd.DataFrame([
    ["SOFR-1B", 0.25,  one_bd,  0.0095,    0.50, 0.20, -0.15],
    ["SOFR-1B", 0.50,  one_bd,  0.0100,    0.50, 0.20, -0.15],
    ["SOFR-1B", 1.00,  one_bd,  0.0105,    0.50, 0.20, -0.15],
    ["SOFR-1B", 2.00,  one_bd,  0.0110,    0.50, 0.20, -0.15]
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

for idx_name, sub in sabr_sofr_1bd.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub
            .pivot(index="AXIS1", columns="AXIS2", values=param)
            .sort_index(axis=0).sort_index(axis=1)
        )
        bu_objs.append(
            Data2D.createDataObject(
                data_type       = param.lower(),
                data_convention = idx_name,
                df              = pivot
            )
        )

In [18]:
bottom_up_build_methods_sofr = []
for param in ("NORMALVOL","BETA","NU","RHO"):
    bottom_up_build_methods_sofr.append({
      "TARGET":          "SOFR-1B",
      "VALUES":          param,
      "INTERPOLATION":   "LINEAR",
      "SHIFT":           0.0,
      "VOL_DECAY_SPEED": 0.2
    })

sabr_dc = DataCollection(bu_objs)
bottom_up_sofr_sabr = SabrModel.from_curve(
    value_date,
    sabr_dc,
    bottom_up_build_methods_sofr,
    yc
)
print("Bottom-up 1BD‐tenor SABR components:", bottom_up_sofr_sabr.components.keys())


Bottom-up 1BD‐tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [ ]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)
ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr,
    {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr},
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)


Overnight Caplet PV (bottom-up): ['USD', np.float64(5370.165454736493)]


In [20]:
ois_cap = ProductOvernightCapFloor(
    effectiveDate="2026-07-01",
    maturityDate ="2027-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1_000_000,
    longOrShort  ="LONG"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr,
    {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr},
    ois_cap
)
ve.calculateValue()
print("Overnight Cap PV (bottom-up):", ve.value)

Overnight Cap PV (bottom-up): ['USD', np.float64(19999.494118054245)]


In [21]:
def price(prod):
    ve = ValuationEngineRegistry().new_valuation_engine(
        bottom_up_sofr_sabr, {"SABR_METHOD":"bottom-up"}, prod
    )
    ve.calculateValue()
    return float(ve.value[1])

caplet_L = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01", termOrEnd="3M", index="SOFR-1B",
    compounding="COMPOUND", optionType="CAP", strike=0.018,
    notional=1_000_000, longOrShort="LONG"
)
caplet_S = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01", termOrEnd="3M", index="SOFR-1B",
    compounding="COMPOUND", optionType="CAP", strike=0.018,
    notional=1_000_000, longOrShort="SHORT"
)
print(price(caplet_L), price(caplet_S))


5370.165454736493 -5370.165454736493


In [22]:
def pv_caplet(K):
    p = ProductOvernightCapFloorlet(
        effectiveDate="2026-07-01", termOrEnd="3M", index="SOFR-1B",
        compounding="COMPOUND", optionType="CAP", strike=K,
        notional=1_000_000, longOrShort="LONG"
    )
    return price(p)

for K in [0.010, 0.016, 0.018, 0.022, 0.030]:
    print(K, pv_caplet(K))


0.01 7323.578316902095
0.016 5856.722284963883
0.018 5370.165454736493
0.022 4408.080386858584
0.03 2612.827718439461


In [24]:
from fixedincomelib.product import ProductOvernightSwap

def pv_capfloor(optionType, K):
    p = ProductOvernightCapFloor(
        effectiveDate="2026-07-01", maturityDate="2027-07-01", frequency="3M",
        index="SOFR-1B", compounding="COMPOUND",
        optionType=optionType, strike=K, notional=1_000_000, longOrShort="LONG"
    )
    ve = ValuationEngineRegistry().new_valuation_engine(
        bottom_up_sofr_sabr, {"SABR_METHOD":"bottom-up"}, p
    )
    ve.calculateValue()
    return float(ve.value[1])

K = 0.018
cap  = pv_capfloor("CAP", K)
floor = pv_capfloor("FLOOR", K)

swap = ProductOvernightSwap(
    effectiveDate="2026-07-01", maturityDate="2027-07-01", frequency="3M",
    overnightIndex="SOFR-1B", spread=0.0, fixedRate=K,
    notional=1_000_000, position="SHORT"   # pay fixed / receive float (in your library)
)
ve_swap = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX":"SOFR-1B"}, swap
)
ve_swap.calculateValue()
swap_pv = float(ve_swap.value[1])

print("cap:", cap, "floor:", floor, "cap-floor:", cap-floor, "swap:", swap_pv, "diff:", (cap-floor) - swap_pv)


cap: 19999.494118054245 floor: 131.55557556270418 cap-floor: 19867.93854249154 swap: 19867.93854249053 diff: 1.0113581083714962e-09
